In [17]:
#import
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
import seaborn as sns
import pandas as pd
import re
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ntorj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
uni = pd.read_csv("crawled_uni.csv")
uni.head()

,Unnamed: 0,name,link,research,field,id
0,0,Prof Sophia Ananiadou,https://www.research.manchester.ac.uk/portal/s...,\nResearch interestsProf. Ananiadou's main con...,cs,412
1,1,Dr Richard Banach,https://www.research.manchester.ac.uk/portal/r...,\nResearch interestsFor accurate and up to dat...,cs,267
2,2,Prof Gavin Brown,https://www.research.manchester.ac.uk/portal/g...,\nResearch interestsMachine Learning.\n,cs,374
3,3,Dr Ke Chen,https://www.research.manchester.ac.uk/portal/k...,\nResearch interestsMy general areas of intere...,cs,170
4,4,Prof Timothy Cootes,https://www.research.manchester.ac.uk/portal/t...,\nResearch interestsMedical Image Analysis usi...,cs,418


In [19]:
X,y =(uni.research, uni.field)

In [20]:
uni['field'].value_counts()

ce    297
es    189
cs     63
Name: field, dtype: int64

In [21]:
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Converting to Lowercase
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [24]:
model = LogisticRegression()
model.fit(X_train, y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
y_pred = model.predict(X_test)

In [26]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[56  1  3]
 [ 5  1  2]
 [ 4  0 38]]
              precision    recall  f1-score   support

          ce       0.86      0.93      0.90        60
          cs       0.50      0.12      0.20         8
          es       0.88      0.90      0.89        42

    accuracy                           0.86       110
   macro avg       0.75      0.65      0.66       110
weighted avg       0.84      0.86      0.84       110

0.8636363636363636


In [27]:
dump(vectorizer, "preprocessor.joblib")
dump(model, "classifier.joblib")

['classifier.joblib']

In [28]:
preprocessor = load("preprocessor.joblib")
model = load("classifier.joblib")

def predict_topic():
  text = input("Enter text to be predicted: ")
  text = preprocessor.transform([text])
  prediction = model.predict(text)[0]
  print("This area is most likely to be in {}." .format(prediction))

In [29]:
predict_topic()

Enter text to be predicted: chemistry
This area is most likely to be in es.
